# General relativity using symbolic computer algebra

## Christoffel symbols and geodesics

In this notebook, we will consider several examples of two-dimensional surfaces. We will symbolically determine the corresponding Christoffel symbols and obtain the differential equations for the geodesic. These differential equations will be solved numerically and the geodesic will be represented graphically. Initial conditions can be modified to observe how the geodesic changes.

## Contents
1. [Cylinder](#cylinder) 
1. [Torus](#torus)
1. [Surface of revolution](#sor)
1. [Sphere](#sphere)

In [ ]:
from gravipy.tensorial import *
from sympy import cos, diag, diff, limit, Matrix, pprint, simplify, sin
import numpy as np
from scipy import integrate, spatial
import k3d 
from ipywidgets import widgets, interact_manual, interact

Later, we will need to display surfaces and provide here a small class. We will subclass `Surface2D` for specific surfaces and, in particular, provide a corresponding method `get_points`.

In [ ]:
class Surface2D:
    def __init__(self, p1min, p1max, dim1, p2min, p2max, dim2, constants=None):
        self.parameters = np.mgrid[p1min:p1max:1j*dim1, p2min:p2max:1j*dim2].T.reshape(-1, 2)
        if constants is not None:
            self.constants = constants
        self.get_points() 
        
    def get_points(self):
        '''set array self.points containing collection of points on the surface

        '''
        raise NotImplementedError
        
    def display(self, wframe=False, opac=1):
        triangulation = spatial.Delaunay(self.parameters)
        plot = k3d.plot()
        points = k3d.mesh(self.points, triangulation.simplices.astype(np.uint32), wireframe=wframe, 
                          opacity=opac, color=0xffc000)
        plot += points
        
        return points

<a id='cylinder'></a>
## Cylinder

We illustrate the strategy used in later examples by considering first a cylinder of unit radius. A point on the cylinder is then specified by the coordinates $(\phi, z)$ with
\begin{align}
 x &= \cos(\phi)\\
 y &= \sin(\phi)\\
 z &= z
\end{align}

In [ ]:
phi, z = symbols('phi z')
x = Coordinates('x', [phi, z])

The parametrization of the surface is used in the method `get_points` to determine the position of the grid points. For illustration, we display the surface right away and invite you to manipulate the figure by rotating or zooming in and out. In this way, we will later be able to take a detailed look at the geodesics.

In [ ]:
class UnitCylinder(Surface2D):
    def __init__(self, dim1, dim2, constants=5):
        super().__init__(0, 2*np.pi, dim1, 0, constants, dim2)
        
    def get_points(self):
        phi, z = self.parameters.T
        self.points = np.array([np.cos(phi),
                                np.sin(phi),
                                z], dtype=np.float32).T

height = 5
cylinder = UnitCylinder(60, 60, height)

In [ ]:
plot_cylinder = k3d.plot()
cylinder.display()

The line element for the cylinder is given by
$$\mathrm{d}s^2 = \mathrm{d}\phi^2+\mathrm{d}z^2.$$
As a consequence, all Christoffel symbols vanish and the equations for the geodesic read
$$\ddot\phi = 0\qquad \ddot z = 0.$$
It is straightforward to solve these two differential equations and we find
\begin{aligned}
\phi(t) &= v_\phi t + \phi_0\\
z(t) &= v_z + z_0.
\end{aligned}
The simplicity of solution is a consequence of the fact that the cylinder, if cut along the $z$-axis, can be flattened to a plane on which the geodesics are given by straight lines.

The following code allows to plot the geodesic obtained for a given value of $\alpha=\mathrm{arctan}(v_z/v_\phi)$. Change the value at the slider to observe how the geodesic changes with $\alpha$.

In [ ]:
def geodesic_cylinder(alpha, height):
    dim_t = 2000
    if alpha == 0:
        t = np.linspace(0, 2*np.pi, dim_t)
        points = np.array([np.cos(t), np.sin(t), np.zeros_like(t)], dtype=np.float32).T
    else:
        t = np.linspace(0, height/np.sin(alpha), dim_t)
        v_phi = np.cos(alpha)
        v_z = np.sin(alpha)
        phi = v_phi*t
        points = np.array([np.cos(phi), np.sin(phi), v_z*t], dtype=np.float32).T
    return points

geodesic = k3d.line(geodesic_cylinder(0, height), color=0x80, width=0.03, shader='mesh')

@interact(alpha=widgets.FloatSlider(value=0, min=0, max=np.pi/2, step=0.01, continuous_update=False))
def updateplot(alpha):
    geodesic.vertices = geodesic_cylinder(alpha, height)
    
plot_cylinder = k3d.plot()
plot_cylinder += cylinder.display()
plot_cylinder += geodesic
plot_cylinder.display()

<a id='torus'></a>
### Torus

Identifying the top and bottom of the cylinder, we arrive at a torus which can be parametrized as
\begin{align}
 x &= (a+b\cos(\theta))\cos(\phi)\\
 y &= (a+b\cos(\theta))\sin(\phi)\\
 z &= b\sin(\theta)
\end{align}
where $a$ and $b$ are the two radii and the angles $\theta$ and $\phi$ run from $0$ to $2\pi$. 

In [ ]:
theta, phi, a, b = symbols('theta phi a b')
x = Coordinates('x', [theta, phi])

We use the parametrization given above to subclass the `Surface2D` class to represent a torus.

In [ ]:
class Torus(Surface2D):
    def __init__(self, dim1, dim2, radii=(1, 1)):
        self.a, self.b = radii
        super().__init__(0, 2*np.pi, dim1, 0, 2*np.pi, dim2)
        
    def get_points(self):
        theta, phi = self.parameters.T
        self.points = np.array([(self.a+self.b*np.cos(theta))*np.cos(phi),
                                (self.a+self.b*np.cos(theta))*np.sin(phi),
                                self.b*np.sin(theta)], dtype=np.float32).T

In order to obtain the differential equations solved by geodesic, we first need to determine the metric tensor.

In [ ]:
r_torus = Matrix([(a + b*cos(theta))*cos(phi),
                 (a + b*cos(theta))*sin(phi),
                 b*sin(theta)])
    
dr_torus = Matrix([diff(r_torus, theta).T, diff(r_torus, phi).T])
g = MetricTensor('g', x, simplify(dr_torus*dr_torus.T))
g(All, All)

There are two non-trivial Christoffel symbols, namely $\Gamma^\theta_{\phi\phi}$ and $\Gamma^\phi_{\theta\phi} = \Gamma^\phi_{\phi\theta}$.

In [ ]:
Ga = Christoffel('Ga', g)
Ga(-All, All, All)

The differential equations solved by the geodesic then read
\begin{align}
\ddot\theta + \frac{(a+b\cos(\theta))\sin(\theta)}{b}\dot\phi^2 &= 0\\
\ddot\phi - 2\frac{b\sin(\theta)}{a+b\cos(\theta)}\dot\theta\dot\phi &= 0
\end{align}

Below, two parameters can be specified to experiment with different initial conditions. `theta0` specifices the initial value of $\theta$ while `alpha` specifies the angle between the components $v_\theta$ and $v_\phi$ of the velocity. The absolute value of the velocity is chosen to one. Note that the two parameters are taken in units of $\pi$.

In [ ]:
def deriv(u, y):
    theta = y[0, :]
    phi = y[1, :]
    v_theta = y[2, :]
    v_phi = y[3, :]
    return [v_theta,
            v_phi,
            -(a+b*np.cos(theta))*np.sin(theta)/b*v_phi**2,
            2*b*np.sin(theta)/(a+b*np.cos(theta))*v_theta*v_phi]

def geodesic_torus(theta0, alpha):
    v_theta0 = np.sin(alpha)/b
    v_phi0 = np.cos(alpha)/(a+b*np.cos(theta0))
    tmin = 0
    tmax = 50
    solution = integrate.solve_ivp(deriv, (tmin, tmax), (theta0, 0, v_theta0, v_phi0),
                                   vectorized=True,
                                   t_eval=np.linspace(tmin, tmax, 1000))
    theta = solution.y[0]
    phi = solution.y[1]
    x = (a+b*np.cos(theta))*np.cos(phi)
    y = (a+b*np.cos(theta))*np.sin(phi)
    z = b*np.sin(theta)
    return np.array([x, y, z], dtype=np.float32).T

a, b = 4, 1
torus = Torus(60, 60, (a, b))
geodesic = k3d.line(geodesic_torus(0, 0), color=0x80, width=0.03, shader='mesh')

@interact(theta0=widgets.FloatSlider(value=0, min=0, max=1, step=0.01),
          alpha=widgets.FloatSlider(value=0, min=0, max=0.5, step=0.01))
def updateplot(theta0, alpha):
    geodesic.vertices = geodesic_torus(theta0*np.pi, alpha*np.pi)

plot_torus = k3d.plot()
plot_torus += torus.display()
plot_torus += geodesic
plot_torus.display()

<a id='sor'></a>
### Surface of revolution

The torus is an example of a surface of revolution, in this case around the $z$ axis. It is instructive to consider another surface of revolution where the distance of the surface from the $z$ axis is modulated. We thus choose the following parametrization
\begin{align}
 x &= (2+\cos(z))\cos(\phi)\\
 y &= (2+\cos(z))\sin(\phi)\\
 z &= z.
\end{align}

In [ ]:
phi, z = symbols('phi z')
x = Coordinates('x', [phi, z])

Let us take a look at this surface:

In [ ]:
class SurfaceOfRevolution(Surface2D):
    def __init__(self, dim1, dim2):
        super().__init__(0, 2*np.pi, dim1, 0, 6*np.pi, dim2)
        
    def get_points(self):
        phi, z = self.parameters.T
        f = 2+np.cos(z)
        self.points = np.array([f*np.cos(phi),
                                f*np.sin(phi), 
                                z], dtype=np.float32).T
        
surfaceOfRevolution = SurfaceOfRevolution(60, 60)
surfaceOfRevolution.display()

Now we proceed very much like in the case of the torus and first determine the metric tensor.

------------------------

In [ ]:
r_sor = Matrix([(2+cos(z))*cos(phi),
                (2+cos(z))*sin(phi),
                z])
    
dr_sor = Matrix([diff(r_sor, phi).T, diff(r_sor, z).T])
g = MetricTensor('g', x, simplify(dr_sor*dr_sor.T))
g(All, All)

There are three different nontrivial Christoffel symbols $\Gamma^\phi_{\phi z}=\Gamma^\phi_{z\phi}$, $\Gamma^z_{\phi\phi}$, and $\Gamma^z_{zz}$.

In [ ]:
Ga = Christoffel('Ga', g)
Ga(-All, All, All)

The differential equations for the geodesic then read
\begin{align}
 \ddot\phi - \frac{2\sin(z)}{2+\cos(z)}\dot\phi\dot z &= 0\\
 \ddot z + \frac{(2+\cos(z))\sin(z)}{\sin^2(z)+1}\dot\phi^2 + \frac{\sin(2z)}{2(\sin^2(z)+1)}\dot z^2 &= 0.
\end{align}

The following code allows to specify the initial height `z0` and the initial angle `alpha` which specifies the components $v_\phi$ and $v_z$ of the initial velocity very much in the same way as was the case for the torus with $v_\theta$ replaced by $v_z$. Again, the parameters are taken in units of $\pi$.

Before playing around with the parameters, take a look at the geodesic obtained for the initially specified values of `z0` and `alpha`. How does the angle of the geodesic with respect to the plane perpendicular to the $z$ axis vary as the geodesic moves through region with different distance from the $z$ axis?

In [ ]:
def deriv(u, y):
    phi = y[0, :]
    z = y[1, :]
    v_phi = y[2, :]
    v_z = y[3, :]
    return [v_phi,
            v_z,
            2*np.sin(z)/(2+np.cos(z))*v_phi*v_z,
            -(2+np.cos(z))*np.sin(z)/(np.sin(z)**2+1)*v_phi**2+0.5*np.sin(2*z)/(np.sin(z)**2+1)*v_z**2]

def geodesic_sor(z0, alpha):
    v_phi0 = np.cos(alpha)/(2+np.cos(z0))
    v_z0 = np.sin(alpha)/np.sqrt(1+np.sin(z0)**2)
    tmin = 0
    tmax = 40
    solution = integrate.solve_ivp(deriv, (tmin, tmax), (0, z0, v_phi0, v_z0),
                                   vectorized=True,
                                   t_eval=np.linspace(tmin, tmax, 4000))
    phi = solution.y[0]
    z = solution.y[1]
    x = (2+np.cos(z))*np.cos(phi)
    y = (2+np.cos(z))*np.sin(phi)
    return np.array([x, y, z], dtype=np.float32).T

sor = SurfaceOfRevolution(60, 60)
geodesic = k3d.line(geodesic_sor(0, 0), color=0x80, width=0.03, shader='mesh')

@interact(z0=widgets.FloatSlider(value=1.05, min=1, max=2, step=0.01),
          alpha=widgets.FloatSlider(value=0, min=0, max=0.5, step=0.01))
def updateplot(z0, alpha):
    geodesic.vertices = geodesic_sor(z0*np.pi, alpha*np.pi)

plot_sor = k3d.plot()
plot_sor += sor.display()
plot_sor += geodesic
plot_sor.display()

The rotational symmetry implies the conservation of the $z$-component of the angular momentum. As a consequence, one finds [Clairaut's relation](https://en.wikipedia.org/wiki/Clairaut%27s_relation) which states that the cosine of the
angle between the geodesic and the plane perpendicular to the $z$ axis scales like $1/r$ for a fixed angular momentum where $r$ is the distance between $z$ axis and surface at the point under consideration.

Try to realize different scenarios:

1. The geodesic runs along a meridian.
1. The geodesic runs along the length of the surface of rotation.
1. The geodesic connects two constrictions.
1. The geodesic is restricted to one waist.

<a id='sphere'></a>
### Sphere

Our last example is the unit sphere with the well-known parametrization
\begin{align}
 x &= \sin(\theta)\cos(\phi)\\
 y &= \sin(\theta)\cos(\phi)\\
 z &= \cos(\theta)
\end{align}
Since by now the strategy to obtain the geodesic should be pretty well known, we keep comments at a minimum.

In [ ]:
theta, phi = symbols('theta phi')
x = Coordinates('x', [theta, phi])

In [ ]:
class UnitSphere(Surface2D):
    def __init__(self, dim1, dim2, constants=[1]):
        super().__init__(0, np.pi, dim1, 0, 2*np.pi, dim2, constants)
        
    def get_points(self):
        theta, phi = self.parameters.T
        self.points = np.array([np.sin(theta)*np.cos(phi),
                                np.sin(theta)*np.sin(phi),
                                np.cos(theta)], dtype=np.float32).T

In [ ]:
r_sphere = Matrix([sin(theta)*cos(phi),
                   sin(theta)*sin(phi),
                   cos(theta)])
    
dr_sphere = Matrix([diff(r_sphere, theta).T, diff(r_sphere, phi).T])
g = MetricTensor('g', x, simplify(dr_sphere*dr_sphere.T))
g(All, All)

In [ ]:
Ga = Christoffel('Ga', g)
Ga(-All, All, All)

The differential equations for the geodesic on a unit sphere thus read
\begin{align}
 \ddot\theta -\frac{1}{2}\sin(2\theta)\dot\phi^2 &= 0\\
 \ddot\phi+\frac{\sin(2\theta)}{\sin^2(\theta)}\dot\theta\dot\phi &= 0.
\end{align}

Due to the rotational symmetry, it is in principle sufficient to prove that the equator of the sphere is defining a geodesic. Here, we allow to change the initial value of $\theta$ by means of the parameter `theta0` taken in terms of $\pi$ in order to demonstrate that a great circle is obtained more generally from the differential equations given above. The equator is shown in red for reference.

In [ ]:
def deriv(u, y):
    theta = y[0, :]
    phi = y[1, :]
    v_theta = y[2, :]
    v_phi = y[3, :]
    return [v_theta,
            v_phi,
            0.5*np.sin(2*theta)*v_phi**2,
            -np.sin(2*theta)/np.sin(theta)**2*v_theta*v_phi]

def geodesic_sphere(theta0):
    v_phi0 = 1/np.sin(theta0)
    tmin = 0
    tmax = 10
    solution = integrate.solve_ivp(deriv, (tmin, tmax), (theta0, 0, 0, v_phi0),
                                   vectorized=True,
                                   t_eval=np.linspace(tmin, tmax, 4000))
    theta = solution.y[0]
    phi = solution.y[1]
    x = np.sin(theta)*np.cos(phi)
    y = np.sin(theta)*np.sin(phi)
    z = np.cos(theta)
    return np.array([x, y, z], dtype=np.float32).T

sphere = UnitSphere(60, 60)
geodesic = k3d.line(geodesic_sphere(0.5), color=0x80, width=0.03, shader='mesh')

@interact(theta0=widgets.FloatSlider(value=0.5, min=0.01, max=0.99, step=0.01))
def updateplot(theta0):
    geodesic.vertices = geodesic_sphere(theta0*np.pi)

plot_sphere = k3d.plot()
plot_sphere += sphere.display()
plot_sphere += k3d.line(geodesic_sphere(0.5*np.pi), color=0x800000, width=0.03, shader='mesh')
plot_sphere += geodesic
plot_sphere.display()